In [39]:
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"



In [50]:
import requests

def query_table(rpc_url, world_address, table_name, columns):
    url = f"{rpc_url}/q"
    query = f"SELECT {', '.join(columns)} FROM {table_name} LIMIT 1;"
    request_body = [
        {
            "address": world_address,
            "query": query,
        }
    ]

    try:
        response = requests.post(url, json=request_body, headers={
            "Content-Type": "application/json",
            "Accept-Encoding": "gzip"
        })
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error querying table '{table_name}': {e}")
        return None

# Example usage
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"

# Known table and columns
table_name = "Inventory"
columns = ["landId", "item", "quantity"]

result = query_table(rpc, world_address, table_name, columns)
print(result)


In [51]:
# cafe = CafeCosmos(rpc=rpc, world_address=world_address)    

# print(cafe.get_inventory_record(4, 110))

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
0


In [52]:
cafe.world.functions.getRecord("Inventory", [4, 110]).call()

MismatchedABI: 
ABI Not Found!
Found 1 element(s) named `getRecord` that accept 2 argument(s).
The provided arguments are not valid.
Provided argument types: (str,int,int)
Provided keyword argument types: {}

Tried to find a matching ABI element named `getRecord`, but encountered the following problems:
Signature: getRecord(bytes32,bytes32[]), type: function
Argument 1 value `Inventory` is not compatible with type `bytes32`.
Argument 2 value `[4, 110]` is not compatible with type `bytes32[]`.
Signature: getRecord(bytes32,bytes32[],bytes32), type: function
Expected 3 argument(s) but received 2 argument(s).


In [53]:
    def get_inventory_balance(self, land_id, item_id):
        # Convert Inventory to bytes32
        table_id = Web3.to_bytes(text="Inventory").rjust(32, b'\0')
        
        # Prepare key tuple
        key_tuple = [
            Web3.to_bytes(land_id).rjust(32, b'\0'),
            Web3.to_bytes(item_id).rjust(32, b'\0')
        ]
        
        # Field layout (you may need to adjust this based on your specific implementation)
        field_layout = Web3.to_bytes(hexstr="0x0000000000000000000000000000000000000000000000000000000000000060")

        try:
            # Call the getRecord function
            static_data, encoded_lengths, dynamic_data = self.world.functions.getRecord(
                table_id, key_tuple, field_layout
            ).call()

            # Decode the static data (assuming uint256 for quantity)
            quantity = decode(['uint256'], static_data)[0]

            return quantity
        except Exception as e:
            print(f"Error getting inventory: {e}")
            return None

    # Example usage
    land_id = 4
    item_id = 110
    quantity = get_inventory_balance(land_id, item_id)

    if quantity is not None:
        print(f"Inventory balance for land {land_id}, item {item_id}: {quantity}")

TypeError: get_inventory_balance() missing 1 required positional argument: 'item_id'

In [27]:
import requests

def query_table(rpc_url, world_address, table_name, columns):
    url = f"{rpc_url}/q"
    query = f"SELECT {', '.join(columns)} FROM {table_name} LIMIT 1;"
    request_body = [
        {
            "address": world_address,
            "query": query,
        }
    ]

    try:
        response = requests.post(url, json=request_body, headers={
            "Content-Type": "application/json",
            "Accept-Encoding": "gzip"
        })
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error querying table '{table_name}': {e}")
        return None

# Example usage
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"

# Known table and columns
table_name = "Inventory"
columns = ["landId", "item", "quantity"]

result = query_table(rpc, world_address, table_name, columns)
print(result)


[{'jsonrpc': '2.0', 'error': {'code': -32600, 'message': 'invalid JSON-RPC version'}, 'id': None}]


In [30]:
import requests

def query_table(rpc_url, world_address, table_name, columns):
    url = f"{rpc_url}/q"
    query = f"SELECT {', '.join(columns)} FROM {table_name} LIMIT 1;"
    request_body = [
        {
            "address": world_address,
            "query": query,
        }
    ]

    try:
        # Use plain JSON array for the payload
        response = requests.post(url, json=request_body, headers={
            "Content-Type": "application/json",
        })
        response.raise_for_status()

        # Parse and return the response
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error querying table '{table_name}': {e}")
        return None

# Example usage
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"

# Known table and columns
table_name = "Inventory"
columns = ["landId", "item", "quantity"]

result = query_table(rpc, world_address, table_name, columns)
print(result)


Result: [{'jsonrpc': '2.0', 'error': {'code': -32600, 'message': 'invalid JSON-RPC version'}, 'id': None}]


In [62]:
import re
import requests


def parse_mud_config(file_path: str):
    with open(file_path, "r") as f:
        content = f.read()
    
    # Extract tables and schemas
    table_pattern = r"(\w+):\s*{\s*schema:\s*{([^}]+)},\s*key:\s*\[([^]]+)]"
    matches = re.findall(table_pattern, content)

    tables = {}
    for table_name, schema_block, keys in matches:
        schema = {
            k.strip(): v.strip().strip('"')
            for k, v in re.findall(r"(\w+):\s*\"?(\w+\[\]?|\w+)\"?", schema_block)
        }
        keys = [k.strip().strip('"') for k in keys.split(",")]
        tables[table_name] = {"schema": schema, "key": keys}

    return tables


class BaseTable:
    RESERVED_SQL_KEYWORDS = {"exists"}  # Add more keywords if needed

    def __init__(self, sdk, table_name, schema, keys):
        self.sdk = sdk
        self.table_name = table_name
        self.schema = schema
        self.keys = keys

    def _escape_column_name(self, column_name):
        """Escape column names that are reserved SQL keywords."""
        return f'"{column_name}"' if column_name in self.RESERVED_SQL_KEYWORDS else column_name

    def get(self, limit=1000, **filters):
        """
        Query the table with optional filters.

        Args:
            limit (int): Maximum number of rows to retrieve. Default is 1000.
            **filters: Key-value pairs to filter the query.

        Returns:
            List[Dict[str, Any]]: A list of records from the table.
        """
        # Determine the columns to select based on whether the keys are filtered
        if any(key in filters for key in self.keys):
            # Exclude keys from SELECT if they are filtered
            select_columns = ", ".join(
                self._escape_column_name(col)
                for col in self.schema.keys()
                if col not in self.keys
            )
        else:
            # Include all columns if no keys are filtered
            select_columns = ", ".join(
                self._escape_column_name(col) for col in self.schema.keys()
            )

        # Construct the WHERE clause
        where_clause = " AND ".join(
            f"{self._escape_column_name(key)}={repr(value)}" for key, value in filters.items()
        )

        query = f"SELECT {select_columns} FROM {self.table_name}"
        if where_clause:
            query += f" WHERE {where_clause}"
        query += f" LIMIT {limit};"

        payload = [{"address": self.sdk.address, "query": query}]
        response = self.sdk.post(payload)
        return self._parse_response(response)

    def _parse_response(self, response):
        if "result" not in response or not response["result"]:
            return None  # Return None if there are no results

        results = response["result"][0]
        if not results:  # Check if the results array is empty
            return None

        headers, *rows = results
        return [dict(zip(headers, row)) for row in rows]


class TableRegistry:
    def __init__(self, sdk):
        self.sdk = sdk

    def register_table(self, table_name, schema, keys):
        """
        Dynamically create and register a table as an attribute of the registry.
        """
        table_class = type(table_name, (BaseTable,), {})
        table_instance = table_class(self.sdk, table_name, schema, keys)
        setattr(self, table_name, table_instance)


class MUDIndexerSDK:
    def __init__(self, base_url, address, config_path):
        self.base_url = base_url
        self.address = address
        self.tables = TableRegistry(self)

        # Parse and register tables from the configuration file
        tables = parse_mud_config(config_path)
        for table_name, table_info in tables.items():
            self.tables.register_table(table_name, table_info["schema"], table_info["key"])

    def post(self, payload):
        response = requests.post(self.base_url, json=payload, headers={"Content-Type": "application/json"})
        if response.status_code != 200:
            raise Exception(f"Request failed with status {response.status_code}: {response.text}")
        return response.json()


In [63]:
# Initialize the SDK with the configuration file
config_path = "../packages/cafecosmos-contracts/mud.config.ts"
sdk = MUDIndexerSDK(
    base_url="https://indexer.mud.garnetchain.com/q",
    address="0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE",
    config_path=config_path
)

# Query ActiveStoves without filters
stoves = sdk.tables.ActiveStoves.get()
print(stoves)

# Query ActiveStoves with a filter
filtered_stoves = sdk.tables.ActiveStoves.get(itemId=79)
print(filtered_stoves)

[{'itemid': '79', 'totalactivestoves': '7'}]
[{'totalactivestoves': '7'}]


In [58]:



# Accessing a table dynamically
recipes = sdk.tables.CraftingRecipe.get(output=110)
print(recipes)

[{'outputquantity': '1', 'xp': '1', 'exists': True, 'inputs': ['20', '23'], 'quantities': ['3', '2']}]


In [60]:
sdk.tables.ActiveStoves.get(itemId=79)

[{'totalactivestoves': '7'}]